In [ ]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.metrics import *
import datetime as dt


import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

In [ ]:
df_train = pd.read_csv('/kaggle/input/tabular-playground-series-jul-2021/train.csv')
df_test = pd.read_csv('/kaggle/input/tabular-playground-series-jul-2021/test.csv')
df_train.date_time = df_train.date_time.astype('datetime64[ns]')
df_test.date_time = df_test.date_time.astype('datetime64[ns]')

In [ ]:
df_train

In [ ]:
df_test

In [ ]:
df_train.describe().T

In [ ]:
print(df_train.target_carbon_monoxide.value_counts())
print(df_train.target_benzene.value_counts())
print(df_train.target_nitrogen_oxides.value_counts())


target_benzeneが0.1の場合は、データが正常に取得できていないと思われる。

In [ ]:
df_train['year'] = df_train.date_time.dt.year
df_train['month'] = df_train.date_time.dt.month
df_train['day'] = df_train.date_time.dt.day
df_train['week_day'] = df_train.date_time.dt.weekday
df_train['hour'] = df_train.date_time.dt.hour
df_train

df_test['year'] = df_test.date_time.dt.year
df_test['month'] = df_test.date_time.dt.month
df_test['day'] = df_test.date_time.dt.day
df_test['week_day'] = df_test.date_time.dt.weekday
df_test['hour'] = df_test.date_time.dt.hour

# 時間による周期性の確認

In [ ]:
df_list = []
for i in range(1,30):
    df = df_train[(df_train.month == 12) & (df_train.day == i)]
    df_list.append(df)

df_list[0].head()

In [ ]:
fig, ax = plt.subplots(6, 5,figsize=(16.0, 12.0))

for i in range(0,29):
    plt.subplot(6,5,i+1)
    ax[i//5][i-(i//5)*5].plot(df_list[i].hour,df_list[i].target_carbon_monoxide)
    ax[i//5][i-(i//5)*5].set_title(str(df_list[i].month.iloc[0]) + '-' + str(df_list[i].day.iloc[0]))

plt.show()

In [ ]:
fig, ax = plt.subplots(6, 5,figsize=(16.0, 12.0))

for i in range(0,29):
    plt.subplot(6,5,i+1)
    ax[i//5][i-(i//5)*5].plot(df_list[i].hour,df_list[i].target_benzene)
    ax[i//5][i-(i//5)*5].set_title(str(df_list[i].month.iloc[0]) + '-' + str(df_list[i].day.iloc[0]))

plt.show()

In [ ]:
fig, ax = plt.subplots(6, 5,figsize=(16.0, 12.0))

for i in range(0,29):
    plt.subplot(6,5,i+1)
    ax[i//5][i-(i//5)*5].plot(df_list[i].hour,df_list[i].target_nitrogen_oxides)
    ax[i//5][i-(i//5)*5].set_title(str(df_list[i].month.iloc[0]) + '-' + str(df_list[i].day.iloc[0]))

plt.show()

## 完全な周期性はないが、3つの値とも少なくとも早朝の時間（3-5時ぐらい）は値が低いことは見て取れるので、弱いながらも時間による周期性はある。

# 日付による周期性の有無の確認

In [ ]:
df_day = df_train.groupby('week_day').mean()
df_day.head()

In [ ]:
plt.plot(df_day.index,df_day.target_carbon_monoxide)
plt.title('target_carbon_monoxide')
plt.show()

plt.plot(df_day.index,df_day.target_benzene)
plt.title('target_benzene')
plt.show()

plt.plot(df_day.index,df_day.target_nitrogen_oxides)
plt.title('target_nitrogen_oxides')
plt.show()


## 土日には、ターゲット3つとも少なくなる傾向がある

In [ ]:
df_month = df_train.groupby('month').mean()
df_month

In [ ]:
plt.plot(df_month.index,df_month.target_carbon_monoxide)
plt.title('target_carbon_monoxide')
plt.show()

plt.plot(df_month.index,df_month.target_benzene)
plt.title('target_benzene')
plt.show()

plt.plot(df_month.index,df_month.target_nitrogen_oxides)
plt.title('target_nitrogen_oxides')
plt.show()

plt.plot(df_month.index,df_month.sensor_1)
plt.title('sensor_1')
plt.show()

plt.plot(df_month.index,df_month.sensor_2)
plt.title('sensor_2')
plt.show()

plt.plot(df_month.index,df_month.sensor_3)
plt.title('sensor_3')
plt.show()

plt.plot(df_month.index,df_month.sensor_4)
plt.title('sensor_4')
plt.show()

plt.plot(df_month.index,df_month.sensor_5)
plt.title('sensor_5')
plt.show()

plt.plot(df_month.index,df_month.deg_C)
plt.title('deg_C')
plt.show()

plt.plot(df_month.index,df_month.relative_humidity)
plt.title('relative_humidity')
plt.show()

plt.plot(df_month.index,df_month.absolute_humidity)
plt.title('absolute_humidity')
plt.show()

3つのターゲット全てにおいて、８月が最も小さく11月が大きい  
特にnitrogen_oxidesにおいては、変動の幅が大きい  
全体的な季節変動と、気温、湿度、センサー類に大きな相関はなさそう

## まとめ
週、時間については、明らかな周期性が存在する。
季節性の変動は、存在しているがターゲットデータである、1,2月のデータが存在しない点が問題  
8月を0として、量的変数として、月データを設定することで特徴を掴むことができるか確認  
ベンゼンが0.1の時はベンゼンのデータが取れていないと思われる。  
ベンゼンの学習データとしては、削除する必要がある

In [ ]:
# 1-2月を12月と3月の間にしたいので、月データを8月始まりに変更
# 1月を12月、2月を3月に変更

def transform_month(month):
    if month > 7:
        month = month - 7
    else:
        month = month + 5
        
    
    if month == 6:
        month = 5
    elif month == 7:
        month = 8
    
    return month


In [ ]:
df_train['month'] = df_train['month'].apply(transform_month)
df_test['month'] = df_test['month'].apply(transform_month)

In [ ]:
# 1月分のデータが1件で1月データを代表してしまうため削除する

df_train = df_train.drop(7110)

In [ ]:
!pip install pycaret

In [ ]:
from pycaret.regression import *
import shap

# Predict carbon_monoxide

In [ ]:
train_c = df_train[['deg_C', 'relative_humidity', 'absolute_humidity',
       'sensor_1', 'sensor_2', 'sensor_3', 'sensor_4', 'sensor_5',
       'target_carbon_monoxide','hour', 'week_day','month']]

test = df_test[['deg_C', 'relative_humidity', 'absolute_humidity',
       'sensor_1', 'sensor_2', 'sensor_3', 'sensor_4', 'sensor_5',
       'hour', 'week_day','month']]


In [ ]:
reg_c = setup(data=train_c,target='target_carbon_monoxide',session_id=12,numeric_features=['month'])

In [ ]:
# モデルの比較を実施
compare_models()

In [ ]:
cat = create_model('catboost')
# tuned_cat = tune_model(cat)

In [ ]:
final_c = finalize_model(cat)


In [ ]:
pred_c = predict_model(final_c, data=test)

# Predict Benzene

In [ ]:
train_b = df_train[['deg_C', 'relative_humidity', 'absolute_humidity',
       'sensor_1', 'sensor_2', 'sensor_3', 'sensor_4', 'sensor_5',
       'target_benzene','hour', 'week_day','month']]

# ベンゼンが0.1のものはデータが取れていないと思われる
train_b = train_b.drop(train_b.index[train_b.target_benzene == 0.1],axis=0)

In [ ]:
reg_b = setup(data=train_b,target='target_benzene',session_id=12,numeric_features=['month'])

In [ ]:
# モデルの比較を実施
compare_models()

In [ ]:
cat_b = create_model('catboost')
#tuned_cat_b = tune_model(cat_b)

In [ ]:
final_b = finalize_model(cat_b)
pred_b = predict_model(final_b, data=test)

# Nitrogen oxides

In [ ]:
train_n = df_train[['deg_C', 'relative_humidity', 'absolute_humidity',
       'sensor_1', 'sensor_2', 'sensor_3', 'sensor_4', 'sensor_5',
       'target_nitrogen_oxides','hour', 'week_day','month']]

In [ ]:
reg_n = setup(data=train_n,target='target_nitrogen_oxides',session_id=12,numeric_features=['month'])

In [ ]:
# モデルの比較を実施
compare_models()

In [ ]:
et_n = create_model('et')
# tuned_cat_n = tune_model(cat_n)

In [ ]:
final_n = finalize_model(et_n)
pred_n = predict_model(final_n, data=test)

In [ ]:
submission = pd.read_csv('/kaggle/input/tabular-playground-series-jul-2021/sample_submission.csv')

In [ ]:
submission.head()

In [ ]:
submission.target_carbon_monoxide = pred_c.Label
submission.target_benzene = pred_b.Label
submission.target_nitrogen_oxides = np.maximum(pred_n.Label,0)

In [ ]:
submission

In [ ]:
submission.describe()

In [ ]:
submission.to_csv('catboost_etr_submission2.csv',index=False)

score:0.26220